In [1]:
import numpy as np
import torch 
import yaml
import torch.nn as nn
import math 

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import src.models as models
import src.dataset as ds
import src.train_pcdae as train_pcdae
import src.inference_pcdae as inference
import src.data_preparation as data_prep

In [2]:

config_file = "config_pcdae.yaml"
device = torch.device('cpu')

try:
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    print("Error: The file 'config.yaml' was not found.")
    exit()
except yaml.YAMLError as e:
    print(f"Error parsing YAML file: {e}")
    exit()
    


In [3]:
def inference_test(seed):
    
    ##! fix the seeds
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    ###! load data and trained model
    data_dict = data_prep.prepare_data_model(seed, config['data']["data_path"], config['training']["ratio_test_val_train"])
    X_test_scaled, y_test_scaled = data_dict['test']
    scaler_X, scaler_Y = data_dict['scalers']
    batch_size = config['training']['batch_size']

    test_dataset = ds.LTPDataset(X_test_scaled, y_test_scaled)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    ###* define the PCDAE architecture
    x_dim = config['model']['x_dim']
    y_dim = config['model']['y_dim']
    hidden_dim = config['model']['hidden_dim']
    
    T1_T = config['inference']['T1_T']
    T1_K = config['inference']['T1_K']
    T2_T = config['inference']['T2_T']
    T2_K = config['inference']['T2_K']
    
    pcdae = models.PCDAE(x_dim=x_dim, y_dim=y_dim, hidden_dim=hidden_dim).to(device)
    pcdae.load_state_dict(torch.load(f"models_pcdae_saved/pcdae_inference_{seed}.pth"))
    
    ##! presented in the reverse order
    T1_vec = torch.linspace(1, 0, T1_T)
    T2_vec = torch.linspace(1, 0, T2_T)
    
    s_noise = float(config['model']['s_noise'])
    sigma_max = float(config['model']['sigma_max'])
    
    T1_schedule = sigma_max * torch.sin((T1_vec + s_noise)/(1.0 + s_noise) * torch.pi / 2.0).to(device)
    T2_schedule = sigma_max * torch.sin((T1_vec + s_noise)/(1.0 + s_noise) * torch.pi / 2.0).to(device)
    
    ###! EBM sampling

    test_init_loss = 0
    test_T1_loss = 0
    test_T2_loss = 0
    
    constant_init_loss = 0
    constant_T_loss = 0
    constant_Tadapt_loss = 0

    for i, (x, y) in enumerate(test_dataloader):
        
        y_init = torch.randn_like(y)
        y_T1 = inference.inference_scheduled_ODE(pcdae, x, y_init, 
                            noise_schedule=T1_schedule, steps_per_level=T1_K, eps_clip=None)
        
        y_T2 = inference.inference_scheduled_ODE(pcdae, x, y_init, 
                            noise_schedule=T2_schedule, steps_per_level=T2_K, eps_clip=None)
        
        
        constant_init_loss += inference.get_constraints_scalar(x, y_init, scaler_X, scaler_Y).pow(2).mean()
        constant_T_loss += inference.get_constraints_scalar(x, y_T1, scaler_X, scaler_Y).pow(2).mean()
        constant_Tadapt_loss += inference.get_constraints_scalar(x, y_T2, scaler_X, scaler_Y).pow(2).mean()
        
        loss_init = nn.MSELoss()(y_init, y)
        loss_T1 = nn.MSELoss()(y_T1, y)
        loss_T2 = nn.MSELoss()(y_T2, y)
        
        test_init_loss += loss_init.item()
        test_T1_loss += loss_T1.item()
        test_T2_loss += loss_T2.item()
        
        print("pro: ", (i+1)/len(test_dataloader))

    test_init_loss /= len(test_dataloader)
    test_T1_loss /= len(test_dataloader)
    test_T2_loss /= len(test_dataloader)
    
    constant_init_loss /= len(test_dataloader)
    constant_T_loss /= len(test_dataloader)
    constant_Tadapt_loss /= len(test_dataloader)
    
    return np.sqrt(test_init_loss), np.sqrt(test_T1_loss), np.sqrt(test_T2_loss), pcdae, constant_init_loss, constant_T_loss, constant_Tadapt_loss



In [4]:

# seed_vec = np.array([1, 41, 81, 121, 161, 201, 241, 281, 321, 361])
seed_vec = np.array([1])

rmse_init_vec = []
rmse_T1_vec = []
rmse_T2_vec = []

for i, seed in enumerate(seed_vec):
    
    val_init, val_T1, val_T2, pcdae, constraint_init, constraint_T, constraint_Tadapt = inference_test(seed)
    
    rmse_init_vec.append(val_init)
    rmse_T1_vec.append(val_T1)
    rmse_T2_vec.append(val_T2)
    
    print("seed: ", seed)
    


pro:  0.1
pro:  0.2
pro:  0.3
pro:  0.4
pro:  0.5
pro:  0.6
pro:  0.7
pro:  0.8
pro:  0.9
pro:  1.0
seed:  1


In [5]:

rmse_init_arr = np.array(rmse_init_vec)
rmse_T1_arr = np.array(rmse_T1_vec)
rmse_T2_arr = np.array(rmse_T2_vec)


print(np.mean(rmse_init_vec))
print(np.mean(rmse_T1_vec))
print(np.mean(rmse_T2_vec))
print()

print(np.sqrt(constraint_init))
print(np.sqrt(constraint_T))
print(np.sqrt(constraint_Tadapt))

1.4022851448245575
0.022653907075902844
0.022657518445107442

tensor(4.4816)
tensor(0.0340)
tensor(0.0340)


/var/folders/l6/sz7h_28d1vn668qy7qlhpvzw0000gn/T/ipykernel_30722/1763533938.py:11: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print(np.sqrt(constraint_init))
/var/folders/l6/sz7h_28d1vn668qy7qlhpvzw0000gn/T/ipykernel_30722/1763533938.py:12: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print(np.sqrt(constraint_T))
/var/folders/l6/sz7h_28d1vn668qy7qlhpvzw0000gn/T/ipykernel_30722/1763533938.py:13: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print(np.sqrt(constraint_Tadapt))


In [6]:
# 1.4022851448245575
# 0.022653040582816645
# 0.022657518059754155

In [7]:
# 0.01311752153355084

In [8]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())


print(count_parameters(pcdae))

11305


In [9]:
# 1.4364558830118268
# 0.031021168598093285
# 0.01793517113448239